In [1]:
# # TechVidvan Vehicle counting and Classification

# # Import necessary packages

# import cv2
# import cv2
# import csv
# import dlib
# import math
# import time
# import collections
# import numpy as np
# from Py_files.tracker import *

# # Initialize Tracker
# tracker = EuclideanDistTracker()

# # Initialize the videocapture object
# cap = cv2.VideoCapture('Videos/video.mp4')
# input_size = 320

# # Detection confidence threshold
# confThreshold =0.2
# nmsThreshold= 0.2

# font_color = (0, 0, 255)
# font_size = 0.5
# font_thickness = 2

# # Middle cross line position
# middle_line_position = 550
# up_line_position = middle_line_position - 15
# down_line_position = middle_line_position + 15


# # Store Coco Names in a list
# classesFile = "yolo/coco.names"
# classNames = open(classesFile).read().strip().split('\n')
# print(classNames)
# print(len(classNames))

# # class index for our required detection classes
# required_class_index = [2, 3, 5, 7]

# detected_classNames = []

# ## Model Files
# modelConfiguration = 'yolo/yolov4-custom.cfg'
# modelWeigheights = 'yolo/yolov4.weights'

# # configure the network model
# net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

# # Configure the network backend

# net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# # Define random colour for each class
# np.random.seed(42)
# colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')


# # Function for finding the center of a rectangle
# def find_center(x, y, w, h):
#     x1=int(w/2)
#     y1=int(h/2)
#     cx = x+x1
#     cy=y+y1
#     return cx, cy
    
# # List for store vehicle count information
# temp_up_list = []
# temp_down_list = []
# up_list = [0, 0, 0, 0]
# down_list = [0, 0, 0, 0]

# # List position
# # past_pos = 

# # Function for count vehicle
# def count_vehicle(box_id, img):

#     x, y, w, h, id, index = box_id

#     # Find the center of the rectangle for detection
#     center = find_center(x, y, w, h)
#     ix, iy = center
    
#     # Find the current position of the vehicle
#     if (iy > up_line_position) and (iy < middle_line_position):

#         if id not in temp_up_list:
#             temp_up_list.append(id)

#     elif iy < down_line_position and iy > middle_line_position:
#         if id not in temp_down_list:
#             temp_down_list.append(id)
            
#     elif iy < up_line_position:
#         if id in temp_down_list:
#             temp_down_list.remove(id)
#             up_list[index] = up_list[index]+1

#     elif iy > down_line_position:
#         if id in temp_up_list:
#             temp_up_list.remove(id)
#             down_list[index] = down_list[index] + 1

#     # Draw circle in the middle of the rectangle
#     cv2.circle(img, center, 2, (0, 0, 255), -1)  # end here
#     # print(up_list, down_list)

# # Function for finding the detected objects from the network output
# def postProcess(outputs,img):
#     global detected_classNames 
#     height, width = img.shape[:2]
#     boxes = []
#     classIds = []
#     confidence_scores = []
#     detection = []
#     for output in outputs:
#         for det in output:
#             scores = det[5:]
#             classId = np.argmax(scores)
#             confidence = scores[classId]
#             if classId in required_class_index:
#                 if confidence > confThreshold:
#                     # print(classId)
#                     w,h = int(det[2]*width) , int(det[3]*height)
#                     x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
#                     boxes.append([x,y,w,h])
#                     classIds.append(classId)
#                     confidence_scores.append(float(confidence))

#     # Apply Non-Max Suppression
#     indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
#     # print(classIds)
#     for i in indices.flatten():
#         x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
#         # print(x,y,w,h)

#         color = [int(c) for c in colors[classIds[i]]]
#         name = classNames[classIds[i]]
#         detected_classNames.append(name)
#         # Draw classname and confidence score 
#         cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
#                   (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

#         # Draw bounding rectangle
#         cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
#         detection.append([x, y, w, h, required_class_index.index(classIds[i])])

#     # Update the tracker for each object
#     boxes_ids = tracker.update(detection)
#     for box_id in boxes_ids:
#         count_vehicle(box_id, img)


# def realTime():
#     width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
#     height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
#     size = (width, height)
#     fourcc = cv2.VideoWriter_fourcc(*'XVID')
#     out = cv2.VideoWriter('your_video.avi', fourcc, 20.0, size)
    
#     while True:
#         success, img = cap.read()
#         img = cv2.resize(img,(0,0),None,1,1)
#         print(img.shape)
#         ih, iw, channels = img.shape
#         blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

#         # Set the input of the network
#         net.setInput(blob)
#         layersNames = net.getLayerNames()
#         outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
#         # Feed data to the network
#         outputs = net.forward(outputNames)
    
#         # Find the objects from the network output
#         postProcess(outputs,img)

#         # Draw the crossing lines

#         cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 2)
#         cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 2)
#         cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 2)

#         # Draw counting texts in the frame
#         cv2.putText(img, "Up", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#         cv2.putText(img, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#         cv2.putText(img, "Car:        "+str(up_list[0])+"     "+ str(down_list[0]), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#         cv2.putText(img, "Motorbike:  "+str(up_list[1])+"     "+ str(down_list[1]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#         cv2.putText(img, "Bus:        "+str(up_list[2])+"     "+ str(down_list[2]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#         cv2.putText(img, "Truck:      "+str(up_list[3])+"     "+ str(down_list[3]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

#         # Show the frames
#         out.write(img)
#         cv2.imshow('Output', img)
#         if cv2.waitKey(1) == ord('q'):
#             break

#     # Write the vehicle counting information in a file and save it

#     with open("data.csv", 'w') as f1:
#         cwriter = csv.writer(f1)
#         cwriter.writerow(['Direction', 'car', 'motorbike', 'bus', 'truck'])
#         up_list.insert(0, "Up")
#         down_list.insert(0, "Down")
#         cwriter.writerow(up_list)
#         cwriter.writerow(down_list)
#     f1.close()
#     print("Data saved at 'data.csv'")
#     # Finally realese the capture object and destroy all active windows
#     cap.release()
#     out.release()
#     cv2.destroyAllWindows()


# image_file = 'vehicle.jpg'
# def from_static_image(image):
#     img = cv2.imread(image)
#     blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

#     # Set the input of the network
#     net.setInput(blob)
#     layersNames = net.getLayerNames()
#     outputNames = [(layersNames[i[0] - 1]) for i in net.getUnconnectedOutLayers()]
#     # Feed data to the network
#     outputs = net.forward(outputNames)

#     # Find the objects from the network output
#     postProcess(outputs,img)

#     # count the frequency of detected classes
#     frequency = collections.Counter(detected_classNames)
#     print(frequency)
#     # Draw counting texts in the frame
#     cv2.putText(img, "Car:        "+str(frequency['car']), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#     cv2.putText(img, "Motorbike:  "+str(frequency['motorbike']), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#     cv2.putText(img, "Bus:        "+str(frequency['bus']), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
#     cv2.putText(img, "Truck:      "+str(frequency['truck']), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)


#     cv2.imshow("image", img)

#     cv2.waitKey(0)

#     # save the data to a csv file
#     with open("static-data.csv", 'a') as f1:
#         cwriter = csv.writer(f1)
#         cwriter.writerow([image, frequency['car'], frequency['motorbike'], frequency['bus'], frequency['truck']])
#     f1.close()


# if __name__ == '__main__':
#     realTime()
# #     from_static_image(image_file)

In [2]:
# Import necessary packages

import cv2
import csv
import dlib
import math
import time
import collections
import numpy as np
from Py_files.tracker import *

from Speed_Estimation.speed_estimate import remove_bad_tracker
from Speed_Estimation.speed_estimate import calculate_speed
# from Speed_Estimation.speed_estimate import test_import



car_detect = cv2.CascadeClassifier('Speed_Estimation/car_detect_harrcascade.xml')

# Cai dat tham so : so diem anh / 1 met, o day dang de 1 pixel = 1 met
pixels_per_meter = 22.2
# Cac tham so phuc vu tracking
# frame_idx = 0
# car_number = 0
# fps = 0

# carTracker = {}
# carNumbers = {}
# carStartPosition = {}
# carCurrentPosition = {}
# speed = [None] * 1000

In [3]:
# TechVidvan Vehicle counting and Classification
import cv2
import csv
import dlib
import math
import time
import collections
import numpy as np
from Py_files.tracker import *

from Speed_Estimation.speed_estimate import remove_bad_tracker
from Speed_Estimation.speed_estimate import calculate_speed


# Function for finding the center of a rectangle
def find_center(x, y, w, h):
    x1=int(w/2)
    y1=int(h/2)
    cx = x+x1
    cy=y+y1
    return cx, cy
    
# List for store vehicle count information
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]

# Function for count vehicle
def count_vehicle(box_id, img):

    x, y, w, h, id, index = box_id

    # Find the center of the rectangle for detection
    center = find_center(x, y, w, h)
    ix, iy = center
    
    # Find the current position of the vehicle
    if (iy > up_line_position) and (iy < middle_line_position):

        if id not in temp_up_list:
            temp_up_list.append(id)

    elif iy < down_line_position and iy > middle_line_position:
        if id not in temp_down_list:
            temp_down_list.append(id)
            
    elif iy < up_line_position:
        if id in temp_down_list:
            temp_down_list.remove(id)
            up_list[index] = up_list[index]+1

    elif iy > down_line_position:
        if id in temp_up_list:
            temp_up_list.remove(id)
            down_list[index] = down_list[index] + 1

    # Draw circle in the middle of the rectangle
    cv2.circle(img, center, 2, (0, 0, 255), -1)  # end here
    cv2.putText(img,str(id), center, cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

    
    
    
# Function for finding the detected objects from the network output
def postProcess(outputs,img):
    global detected_classNames 
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
#     print("Do dai outputs")
#     print(len(outputs))
    
    for output in outputs:
#         print('output')
#         print(outputs)
        for det in output:
#             print('det')
#             print(det)
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if classId in required_class_index:
#                 print('det')
#                 print(det)
                if confidence > confThreshold:
                    # print(classId)
                    w,h = int(det[2]*width) , int(det[3]*height)
                    x,y = int((det[0]*width)-w/2) , int((det[1]*height)-h/2)
                    boxes.append([x,y,w,h])
                    classIds.append(classId)
                    confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)
#     print("Độ dài của boxes:", len(boxes))
#     print("indices")
#     print(indices.flatten())
    # print(classIds)
    for i in indices.flatten():
        x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
        # print(x,y,w,h)

        color = [int(c) for c in colors[classIds[i]]]
#         print(color)
        name = classNames[classIds[i]]
        detected_classNames.append(name)
#         colors.append(color)
        # Draw classname and confidence score 
        cv2.putText(img,f'{name.upper()} {int(confidence_scores[i]*100)}%',
                  (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Draw bounding rectangle
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        detection.append([x, y, w, h, required_class_index.index(classIds[i])])
    
    
    # Update the tracker for each object
    boxes_ids = tracker.update(detection)
#     print("detection: ", detection)
#     print("boxes_ids: ", boxes_ids)
    for box_id in boxes_ids:
        count_vehicle(box_id, img)

    return boxes_ids
        
        
def find_by_id(abcd, num_id):
    for abc in abcd:
        if abc[4] == num_id:
            return abc
#             print("heloo")
            break



In [4]:
# Initialize Tracker
tracker = EuclideanDistTracker()


# Initialize the videocapture object
cap = cv2.VideoCapture('Videos/video.mp4')
input_size = 320

# Detection confidence threshold
confThreshold =0.2
nmsThreshold= 0.2

font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 2

# Middle cross line position
middle_line_position = 500
up_line_position = middle_line_position - 25
down_line_position = middle_line_position + 25


# Store Coco Names in a list
classesFile = "yolo/test.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)
print(len(classNames))

# class index for our required detection classes
required_class_index = [2, 3, 5, 7]

detected_classNames = []

## Model Files
modelConfiguration = 'yolo/yolov4-custom.cfg'
modelWeigheights = 'yolo/yolov4.weights'

# configure the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

# Configure the network backend

net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

carTracker = {}


def realTime():
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
    size = (width, height)
    print(size)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('Videos/output_video.avi', fourcc, 15.0, size)
    
    so_frame = 0
    
    speed_by_id = {}
    
    while True:
        success, img = cap.read()
        
        # Start time
        start_time = time.time()

        if img is None:
            print("End video")
            break
            
        
        
        img = cv2.resize(img,(0,0),None,1,1)
        
        ih, iw, channels = img.shape
#         print(img.shape)
        blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

        # Set the input of the network
        net.setInput(blob)
        layersNames = net.getLayerNames()
        names = net.getUnconnectedOutLayers()
        outputNames = [(layersNames[names[0] - 1])]
                
        # Feed data to the network
        outputs = net.forward(outputNames)
        
    
        # Find the objects from the network output
        if so_frame == 0:
            detection = postProcess(outputs,img)
            
        elif so_frame%5==0:
            past_position = detection
            detection = postProcess(outputs,img)
            
            end_time = time.time()
            if not (end_time == start_time):
                fps = 5.0/(end_time - start_time)
                
            for car in detection:
                x1, y1, w1, h1, id1, index1 = car
                pixels_per_meter = 20
                x1, y1, w1, h1, id1, index1 = car
#                 if y1 >=387 and y1<424:
#                     pixels_per_meter = 11
#                 if y1 >=424 and y1<461:
#                     pixels_per_meter = 15
#                 if y1 >=461 and y1<498:
#                     pixels_per_meter = 19
#                 if y1 >=498 and y1<535:
#                     pixels_per_meter = 22.2
#                 if y1 >=535 and y1<572:
#                     pixels_per_meter = 26.6
#                 if y1 >=572 and y1<609:
#                     pixels_per_meter = 31
#                 if y1 >=609 and y1<646:
#                     pixels_per_meter = 35.4
#                 if y1 >=646 and y1<683:
#                     pixels_per_meter = 37
#                 if y1 >=683 and y1<=720:
#                     pixels_per_meter = 40  
#                 dlib_tracker = dlib.correlation_tracker()
#                 dlib_tracker.start_track(img, dlib.rectangle(x, y, x + w, y + h))
#                 carTracker[id1] = dlib_tracker
                
                if (find_by_id(past_position, id1)):
                    x2, y2, w2, h2, id2, index2 = find_by_id(past_position, id1)
                    
                    
                    speed = calculate_speed([x1, y1, w1, h1], [x2, y2, w2, h2],fps)
                    speed = speed / 5
                    speed_by_id[id1] = speed
#                     print('speed: ', speed)
                    if y1 > 350 and y1<680:

                        cen_x, cen_y = find_center(x1, y1, w1, h1)

                        cv2.putText(img, str(int(speed)) + " km/h",
                                (cen_x, cen_y-12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (0, 255, 255), 1)
#                     else if y1>y2 
#                 else:
#                     continue
                
#             for carID in carTracker.keys():
#                 print('carID', carID)
            
            
        else:
            past_position = detection
            detection = postProcess(outputs,img)
            if so_frame > 5:
                for car in detection:
                    x1, y1, w1, h1, id1, index1 = car
                    if y1 > 350:
                        if id1 in tuple(speed_by_id.keys()):
                            speed = speed_by_id[id1]
                            cen_x, cen_y = find_center(x1, y1, w1, h1)

                            cv2.putText(img, str(int(speed)) + " km/h",
                                    (cen_x, cen_y-12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                    (0, 255, 255), 1)
#             end_time = time.time()
#             if not (end_time == start_time):
#                 fps = 1.0/(end_time - start_time)
                
#             for car in detection:
#                 x1, y1, w1, h1, id1, index1 = car
                
# #                 dlib_tracker = dlib.correlation_tracker()
# #                 dlib_tracker.start_track(img, dlib.rectangle(x, y, x + w, y + h))
# #                 carTracker[id1] = dlib_tracker
                
#                 if (find_by_id(past_position, id1)):
#                     x2, y2, w2, h2, id2, index2 = find_by_id(past_position, id1)
                    
                    
#                     speed = calculate_speed([x1, y1, w1, h1], [x2, y2, w2, h2],fps)

# #                     print('speed: ', speed)
#                     if y1 > 350:

#                         cen_x, cen_y = find_center(x1, y1, w1, h1)

#                         cv2.putText(img, str(int(speed)) + " km/h",
#                                 (cen_x, cen_y-12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,
#                                 (0, 255, 255), 1)
                

        # Draw the crossing lines

        cv2.line(img, (0, middle_line_position), (iw, middle_line_position), (255, 0, 255), 2)
        cv2.line(img, (0, up_line_position), (iw, up_line_position), (0, 0, 255), 2)
        cv2.line(img, (0, down_line_position), (iw, down_line_position), (0, 0, 255), 2)

        # Draw counting texts in the frame
        cv2.putText(img, "Up", (110, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
        cv2.putText(img, "Down", (160, 20), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
        cv2.putText(img, "Car:        "+str(up_list[0])+"     "+ str(down_list[0]), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
        cv2.putText(img, "Motorbike:  "+str(up_list[1])+"     "+ str(down_list[1]), (20, 60), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
        cv2.putText(img, "Bus:        "+str(up_list[2])+"     "+ str(down_list[2]), (20, 80), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
        cv2.putText(img, "Truck:      "+str(up_list[3])+"     "+ str(down_list[3]), (20, 100), cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

        
        # Show the frames
        out.write(img)
        
        cv2.imshow('Output', img)
        if cv2.waitKey(1) == ord('q'):
            break
            
        so_frame += 1
        
#         end = time.time()
#         seconds = end - start
#         num_frames = 1;
#         fps  = num_frames / seconds
#         print("Estimated frames per second : {0}".format(fps))





if __name__ == '__main__':
    realTime()


['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck']
8
(1280, 720)


[ WARN:0@0.663] global /io/opencv/modules/dnn/src/dnn.cpp (1483) setUpNet DNN module was not built with CUDA backend; switching to CPU
QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (0x564ab31d3860)

QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (0x564ab31d3860)

QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (0x564ab31d3860)

QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (0x564ab31d3860)

QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (0x564ab31d3860)

QObject::moveToThread: Current thread (0x564ab31d3860) is not the object's thread (0x564abcd4d0b0).
Cannot move to target thread (

In [5]:
import cv2
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)

In [6]:
if 0%5 == 0:
    print(1)

1


In [7]:
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')


In [8]:
major_ver

'4'

In [9]:
a = [[1030, 491, 122, 89, 0, 0], [377, 412, 65, 51, 1, 0], [202, 441, 94, 58, 2, 0], [853, 434, 77, 63, 3, 0], [535, 364, 44, 36, 4, 0], [290, 668, 152, 51, 5, 0], [738, 405, 55, 55, 6, 0], [753, 273, 55, 57, 7, 3], [445, 325, 32, 21, 8, 0], [495, 323, 40, 29, 9, 0], [674, 306, 32, 20, 10, 0], [565, 314, 30, 16, 11, 0], [508, 307, 30, 15, 9, 0], [664, 274, 27, 20, 12, 0], [545, 289, 20, 16, 13, 0]]

In [10]:
def find_by_id(abcd, num_id):
    for abc in abcd:
        if abc[4] == num_id:
            return abc
            print("heloo")
            break



In [11]:
find_by_id(a, 3)

[853, 434, 77, 63, 3, 0]

In [12]:
def test_print(a):
    return a

In [13]:
a = 5
b = 6
ab = test_print([a, b])

In [14]:
print(ab)

[5, 6]


In [15]:
abc = [5, 6]
a_b = test_print(abc)

In [16]:
a_b

[5, 6]

In [17]:
if ab == a_b:
    print(1)
else:
    print(0)

1


In [18]:
a = {}

a[1] = 123
a[2] = 456

In [19]:
print(a)

{1: 123, 2: 456}


In [20]:
a = (3, 2, 0, 6, 5, 7, 4, 1, 8, 9, 10, 11, 18, 16)

b = 300 in a

In [21]:
b

False